# Expectation-Maximization Clustering

**Maximization step**
$$
q_{mk} = \frac{
            \sum_{n=1}^{N} r_{nk}I(t_m \in d_n)
            }{
            \sum_{n=1}^{N} r_{nk}
            }
            \ ;\ \ \alpha_k = \frac{\sum_{n=1}^{N} r_{nk}}{N}
$$
**Expectation step**
$$
r_{nk} = \frac{
            \alpha_k(\prod_{t_m \in d_n} q_{mk})(\prod_{t_m \not\in d_n} (1-q_{mk}))
            }{
            \sum_{k=1}^{K}\alpha_k(\prod_{t_m \in d_n} q_{mk})(\prod_{t_m \not\in d_n} (1-q_{mk}))
            }
$$

where $I(t_m \in d_m) = 1$ if $t_m \in d_m$, $0$ otherwise. $r_{nk}$ is the soft assignment of $d_n$ to $\omega_k$.

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from IPython.display import display, display_html

- `M` one-hot encoding of docs (multivariate Bernoulli distributions for data)
- `N` corpus size
- `K` number of clusters to find
- `m` vocabulary size
- `A` cluster priors
- `Q` tokens assignment to clusters
- `R` documents assignment to clusters

In [2]:
corpus = [
    "sugar milk eggs butter".split(),
    "vanilla eggs apple peach sugar".split(),
    "peach apple sugar milk".split(),
    "meat chicken olive salt butter".split(),
    "meat beef salt pepper".split()
]
vocabulary = list(set([x for y in corpus for x in y]))
M = np.array([[1 if x in y else 0 for x in vocabulary] for y in corpus])
N, K, m = len(corpus), 2, len(vocabulary)
A = np.zeros(K)
Q = np.zeros((m, K))

In [5]:
pd.DataFrame(M, columns=vocabulary)

,milk,eggs,olive,butter,peach,salt,vanilla,chicken,sugar,pepper,apple,meat,beef
0,1,1,0,1,0,0,0,0,1,0,0,0,0
1,0,1,0,0,1,0,1,0,1,0,1,0,0
2,1,0,0,0,1,0,0,0,1,0,1,0,0
3,0,0,1,1,0,1,0,1,0,0,0,1,0
4,0,0,0,0,0,1,0,0,0,1,0,1,1


## Random init $r_{nk}$ (assignment of documents to clusters)

In [6]:
R = np.random.uniform(size=(N, K))
R = R / R.sum(axis=1).reshape(-1, 1)

In [7]:
R

array([[0.37211092, 0.62788908],
       [0.44166064, 0.55833936],
       [0.3363425 , 0.6636575 ],
       [0.72653079, 0.27346921],
       [0.66509896, 0.33490104]])

## Maximization step

$$
q_{mk} = \frac{
            \sum_{n=1}^{N} r_{nk}I(t_m \in d_n)
            }{
            \sum_{n=1}^{N} r_{nk}
            }
            \ ;\ \ \alpha_k = \frac{\sum_{n=1}^{N} r_{nk}}{N}
$$


In [8]:
R[:,0] * M[:,2]

array([0.        , 0.        , 0.        , 0.72653079, 0.        ])

In [10]:
def qmk(token_index, cluster_index):
    num = (R[:,cluster_index] * M[:,token_index]).sum()
    den = R[:,cluster_index].sum()
    return num / den

def ak(cluster_index):
    return R[:,cluster_index].sum() / N

In [11]:
qmk(2, 0)

0.2858395030307042

In [12]:
ak(0)

0.5083487603252166

In [13]:
def maxstep(Q, A):
    for cluster_index in range(K):
        for token_index in range(len(vocabulary)):
            Q[token_index, cluster_index] = qmk(token_index, cluster_index)
        A[cluster_index] = ak(cluster_index)
    return Q / Q.sum(axis=1).reshape(-1, 1), A

In [14]:
Adf = pd.DataFrame(A)
Qdf = pd.DataFrame(Q, index=vocabulary).T
A_styler = Adf.style.set_table_attributes("style='display:inline'").set_caption('A')
Q_styler = Qdf.style.set_table_attributes("style='display:inline'").set_caption('Q')
display_html(A_styler._repr_html_() + Q_styler._repr_html_(), raw=True)

Q, A = maxstep(Q, A)

Adf = pd.DataFrame(A)
Qdf = pd.DataFrame(Q, index=vocabulary).T
A_styler = Adf.style.set_table_attributes("style='display:inline'").set_caption('A')
Q_styler = Qdf.style.set_table_attributes("style='display:inline'").set_caption('Q')
display_html(A_styler._repr_html_() + Q_styler._repr_html_(), raw=True)

A 
 
 
   
 0 
 
 
 
 
 0 
 0.000000 
 
 
 1 
 0.000000 
 
 
 

 
 Q 
 
 
   
 milk 
 eggs 
 olive 
 butter 
 peach 
 salt 
 vanilla 
 chicken 
 sugar 
 pepper 
 apple 
 meat 
 beef 
 
 
 
 
 0 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 1 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000

A 
 
 
   
 0 
 
 
 
 
 0 
 0.508349 
 
 
 1 
 0.491651 
 
 
 

 
 Q 
 
 
   
 milk 
 eggs 
 olive 
 butter 
 peach 
 salt 
 vanilla 
 chicken 
 sugar 
 pepper 
 apple 
 meat 
 beef 
 
 
 
 
 0 
 0.346625 
 0.398852 
 0.719845 
 0.541040 
 0.381094 
 0.688700 
 0.433442 
 0.719845 
 0.375507 
 0.657619 
 0.381094 
 0.688700 
 0.657619 
 
 
 1 
 0.653375 
 0.601148 
 0.280155 
 0.458960 
 0.618906 
 0.311300 
 0.566558 
 0.280155 
 0.624493 
 0.342381 
 0.618906 
 0.311300 
 0.342381

## Expectation step

$$
r_{nk} = \frac{
            \alpha_k(\prod_{t_m \in d_n} q_{mk})(\prod_{t_m \not\in d_n} (1-q_{mk}))
            }{
            \sum_{k=1}^{K}\alpha_k(\prod_{t_m \in d_n} q_{mk})(\prod_{t_m \not\in d_n} (1-q_{mk}))
            }
$$


In [15]:
def prod(cluster_index, document_index):
    p1 = np.product([qmk(i, cluster_index) for i in range(len(vocabulary)) if M[document_index,i] == 1])
    p0 = np.product([1 - qmk(i, cluster_index) for i in range(len(vocabulary)) if M[document_index,i] == 0])
    return A[cluster_index] * p1 * p0

def expectation(R):
    for document_index in range(N):
        total = sum([prod(c, document_index) for c in range(K)])
        for cluster_index in range(K):
            R[document_index, cluster_index] = prod(cluster_index, document_index) / total
    return R / R.sum(axis=1).reshape(-1, 1)

In [16]:
Rdf = pd.DataFrame(R)
display(Rdf)

R = expectation(R)

Rdf = pd.DataFrame(R)
display(Rdf)

,0,1
0,0.372111,0.627889
1,0.441661,0.558339
2,0.336342,0.663658
3,0.726531,0.273469
4,0.665099,0.334901


,0,1
0,0.082202,0.917798
1,0.023779,0.976221
2,0.001201,0.998799
3,0.999991,0.000009
4,0.999980,0.000020


## Put things together

In [17]:
class EMClustering(object):

    def __init__(self, corpus, k=2):
        self.corpus = corpus
        self.vocabulary = list(set([x for y in self.corpus for x in y]))
        self.M = np.array([[1 if x in y else 0 for x in self.vocabulary] for y in self.corpus])
        self.N, self.K, self.m = len(self.corpus), k, len(self.vocabulary)
        self.A = np.zeros(self.K)
        self.Q = np.zeros((self.m, self.K))
        # Init R
        self.R = np.random.uniform(size=(self.N, self.K))
        self.R = self.R / self.R.sum(axis=1).reshape(-1, 1) 

    def qmk(self, token_index, cluster_index):
        num = (self.R[:,cluster_index] * self.M[:,token_index]).sum()
        den = self.R[:,cluster_index].sum()
        return num / den

    def ak(self, cluster_index):
        return self.R[:,cluster_index].sum() / self.N
    
    def maxstep(self):
        for cluster_index in range(self.K):
            for token_index in range(self.m):
                self.Q[token_index, cluster_index] = self.qmk(token_index, cluster_index)
            self.A[cluster_index] = self.ak(cluster_index)
        self.Q = self.Q / self.Q.sum(axis=1).reshape(-1, 1)

    def prod(self, cluster_index, document_index):
        p1 = np.product([self.qmk(i, cluster_index) for i in range(self.m) if self.M[document_index,i] == 1])
        p0 = np.product([1 - self.qmk(i, cluster_index) for i in range(self.m) if self.M[document_index,i] == 0])
        return self.A[cluster_index] * p1 * p0

    def expectation(self):
        for document_index in range(self.N):
            total = sum([self.prod(c, document_index) for c in range(self.K)])
            for cluster_index in range(self.K):
                self.R[document_index, cluster_index] = self.prod(cluster_index, document_index) / total
    
    def fit(self, iterations=10):
        self.maxstep()
        self.expectation()

## Try with different corpora

In [25]:
corpus = [
    "sugar milk eggs butter".split(),
    "vanilla eggs apple peach sugar".split(),
    "peach apple sugar milk".split(),
    "meat chicken olive salt butter".split(),
    "meat beef salt pepper".split()
]

In [26]:
EM = EMClustering(corpus, k=3)
EM.fit(iterations=10)

In [27]:
Adf = pd.DataFrame(EM.A)
Qdf = pd.DataFrame(EM.Q, index=EM.vocabulary).T
A_styler = Adf.style.set_table_attributes("style='display:inline'").set_caption('A')
Q_styler = Qdf.style.set_table_attributes("style='display:inline'").set_caption('Q')
display_html(A_styler._repr_html_() + Q_styler._repr_html_(), raw=True)
Rdf = pd.DataFrame(EM.R)
display(round(Rdf, 2))

A 
 
 
   
 0 
 
 
 
 
 0 
 0.310152 
 
 
 1 
 0.346733 
 
 
 2 
 0.343115 
 
 
 

 
 Q 
 
 
   
 milk 
 eggs 
 olive 
 butter 
 peach 
 salt 
 vanilla 
 chicken 
 sugar 
 pepper 
 apple 
 meat 
 beef 
 
 
 
 
 0 
 0.391591 
 0.344107 
 0.444074 
 0.479636 
 0.216042 
 0.356471 
 0.166584 
 0.444074 
 0.317844 
 0.267072 
 0.216042 
 0.356471 
 0.267072 
 
 
 1 
 0.251556 
 0.353276 
 0.189440 
 0.182083 
 0.433966 
 0.314876 
 0.538764 
 0.189440 
 0.345689 
 0.442885 
 0.433966 
 0.314876 
 0.442885 
 
 
 2 
 0.356854 
 0.302616 
 0.366486 
 0.338281 
 0.349992 
 0.328653 
 0.294652 
 0.366486 
 0.336467 
 0.290043 
 0.349992 
 0.328653 
 0.290043

,0,1,2
0,0.63,0.08,0.29
1,0.01,0.85,0.14
2,0.04,0.64,0.31
3,0.76,0.00,0.24
4,0.27,0.03,0.70


In [28]:
from collections import defaultdict

In [29]:
cluster_description = defaultdict(list)
for i, cluster in enumerate(np.argmax(EM.Q, axis=1)):
    cluster_description[cluster].append(EM.vocabulary[i])
for cluster, words in cluster_description.items():
    print(cluster, words)

0 ['milk', 'olive', 'butter', 'salt', 'chicken', 'meat']
1 ['eggs', 'peach', 'vanilla', 'sugar', 'pepper', 'apple', 'beef']


In [30]:
cluster_docs = defaultdict(list)
for i, cluster in enumerate(np.argmax(EM.R, axis=1)):
    cluster_docs[cluster].append(corpus[i])
for cluster, docs in cluster_docs.items():
    print(cluster)
    for doc in docs:
        print('-', " ".join(doc))

0
- sugar milk eggs butter
- meat chicken olive salt butter
1
- vanilla eggs apple peach sugar
- peach apple sugar milk
2
- meat beef salt pepper
